# This project examines patterns in manufacturing defects

Data is provided by Bosch

## Import and examine the data

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

tic = time.process_time()

training set has 1023747 records

test set has 1183747 records

In [35]:
folder = 'Bosch-Production-Line-Performance/'
batch_size = 70000
train_num_records = 1023748
skiprows = np.random.randint(1, train_num_records+1, size=train_num_records-batch_size)

In [36]:
%%time
# reading only first 10000 rows because of file size and memory limitations

train_categorical_sample = pd.read_csv(folder+'train_categorical.csv', nrows=batch_size, skiprows=skiprows)
train_numeric_sample = pd.read_csv(folder+'train_numeric.csv', nrows=batch_size, skiprows=skiprows)
train_date_sample = pd.read_csv(folder+'train_date.csv', nrows=batch_size, skiprows=skiprows)


# merge categorical and numeric together for total train_features_sample DataFrame
train_feature_sample = pd.merge(train_categorical_sample, train_numeric_sample,on='Id')

<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,57,59,60,62,63,65,66,68,69,71,72,74,75,77,78,80,81,83,84,86,87,89,90,92,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,188,189,190,191,192,193,324,335,337,340,342,344,346,348,351,354,357,360,363,366,369,372,375,378,381,384,387,390,393,396,399,402,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,464,467,470,473,476,479,482,485,488,491,494,497,500,503,506,509,512,515,518,549,576,587,588,589,590,591,592,593,594,595,596,597,611,614,617,620,623,626,629,632,635,638,641,644,647,650,653,656,659,662,665,668,671,674,677,680,683,686,689,692,695,698,701,704,707,710,713,716,719,722,780,842,843,844,845,846,847,848,8

CPU times: user 40.6 s, sys: 7.21 s, total: 47.8 s
Wall time: 47.9 s


## Let's try to create a visualization of start and end for shopfloor

#### Question: Are Line #s and Station #s exclusive?  Or do there exist Station X for both Line A and B?

In [37]:
def extractLSF(string, isDate=False):
    '''
    Given a string in the Bosch Line/Station/Feature format (eg: L0_S0_F26), extract the Line number, Station number, and Feature number
    
    Input: String in Lx_Sy_Fz format
    
    Output:  Tuple of (x, y, z)
    
    Sidenote:  isDate will extract based on Lx_Sy_Dd format instead, returns tuple of (x, y, d)
    '''
    # first assert the format is correct:
    splitStr = string.split('_')
    if isDate:
        prefixList = ['L', 'S', 'D']
    else:
        prefixList = ['L', 'S', 'F']
        
    assert len(splitStr) == 3, 'Not 3 substrings split by "_"s!'
    for i in range(0, len(splitStr)):
        assert splitStr[i][1:].isnumeric(), 'Not numeric following the single-letter prefix!'
        assert splitStr[i][:1] == prefixList[i], 'Not the proper single-letter prefixes!  Did you use a Date format and forget to set isDate=True?'
    
    #now extract numbers:
    tempList = []
    
    for substr in splitStr:
        number = float(substr[1:])
        if number%1 == 0.0:
            tempList.append(int(number))
        else:
            tempList.append(number)
        
    return tuple(tempList)

In [38]:
def getListLSF(inputSeries, isDate=False):
    '''
    Given a series with fields of Bosch style Lx_Sy_Fz: Get the lines, stations, and features (or dates)
    Input:  series (row) in a Bosch-style QC matrix
    Output:  pandas dataframe of lines, stations, and features/dates
    '''
    
    for field in inputSeries.index:
        try:
            myL, myS, myF = extractLSF(field, isDate=isDate)
        except:
            continue
        listL.append(myL)
        listS.append(myS)
        listF.append(myF)
    
    return {'Lines': listL, 'Stations': listS, 'Features': listF}

In [39]:
def getListLSD(inputSeries):
    return getListLSF(inputSeries, isDate=True)

In [40]:
# All L/S/Fs

def get_lsf(lsf_df, isDate=False):
    '''From a Bosch DF, extract all the Line/Station/Feature combinations in the fields
    Outputs a Dataframe of columns Line, Station, Feature'''
    if isDate:
        lsf = pd.DataFrame(columns=['Line', 'Station', 'Date'])
    else:
        lsf = pd.DataFrame(columns=['Line', 'Station', 'Feature'])
    
    for field in lsf_df.columns:
        try:
            lsf = lsf.append(pd.DataFrame([list(extractLSF(field, isDate))], columns=lsf.columns))
        except:
            continue
    
    return lsf

# ----------------------------------------------------

cat_lsf = get_lsf(train_categorical_sample)
num_lsf = get_lsf(train_numeric_sample)
date_lsd = get_lsf(train_date_sample, isDate=True)

In [41]:
all_lsf = pd.concat([cat_lsf, num_lsf])

In [42]:
print(all_lsf.groupby('Line').min())
print(all_lsf.groupby('Line').max())

      Station  Feature
Line                  
0           0        0
1          24      675
2          26     3036
3          29     3315
      Station  Feature
Line                  
0          23      674
1          25     3034
2          28     3314
3          51     4262


#### It does look like the Line #s and Station #s are exclusive!

Looks like:

0. Line 0 encompasses Station 0 through 23
1. Line 1 encompasses Station 24 through 25
2. Line 2 encompasses Station 26 through 28
3. Line 3 encompasses Station 29 through 51

**We'll see in the test below that Line & Features, Station & Features, Line & Date, and Station & Date are ALL exclusive!!!**

In [43]:
def testLSFexclusive(data, groupbyfield, exclusivefield):
    '''Tests if the exclusiveField (eg: Stations) are exclusive to groupbyField (eg: Lines)
    Does this by ensuring there's no overlap in the min/max ranges'''
    mins = data.groupby(groupbyfield).min()[exclusivefield]
    maxs = data.groupby(groupbyfield).max()[exclusivefield]
    
    bool1 = mins[0] == mins.min()
    bool2 = maxs[-1:].iloc[0] == maxs.max()
    bool3 = True
    
    for i in range(0, mins.size-1):
        if maxs[i] >= mins[i+1]: #careful i is last one
            bool3 = False
    
    return bool1*bool2*bool3

In [44]:
print( testLSFexclusive(all_lsf, 'Line', 'Feature') )
print( testLSFexclusive(all_lsf, 'Station', 'Feature') )
print( testLSFexclusive(date_lsd, 'Line', 'Date') )
print( testLSFexclusive(date_lsd, 'Station', 'Date') )

True
True
True
True


## Now we can produce a shop floor picture because we know that lines, stations, features, and dates don't overlap across each other!

In [45]:
# station_line_assignment is a Dictionary of Stations with their respective Lines

station_line_assignment = all_lsf.groupby('Station').min().drop('Feature', axis=1)

tempdict = {-1:-1} #assign Start station as a Start type
for i in station_line_assignment.index:
    tempdict[i] = station_line_assignment.loc[i][0]
tempdict[99] = 99 #assign Pass result station as Result Line
tempdict[100] = 100 #assign Fail result station as Result Line

station_line_assignment = tempdict

In [46]:
# Creating color codes for Nodes:  Different Lines, Outputs, and Starting Point all have different colors:

#-1=start, 999999=Result, everything else=Line#

nodeShapeDict = {0:'diamond', 1:'star', 2:'square', 3:'triangle', -1:'circle', 99:'circle', 100:'circle'}

#### Creating arguments to feed in Network.add_nodes

In [47]:
nodesList = list(station_line_assignment.keys())
startendlabels = {-1:'START', 99:'PASS', 100:'FAIL'}
nodesLabelList = [startendlabels[station] if station in startendlabels.keys() else 'S{}'.format(station) for station in nodesList]
#nodesSizeList = # <--- temporary, I will create this list in few lines of code down
nodesShapeList = [nodeShapeDict[station_line_assignment[station]] for station in nodesList]

In [48]:
# create new dataframe to ready to get all connections
# this dataframe shows stations only in the Field Names... stations are the only thing I need for connections

dateColsStationOnly = ['Id']
dateColsStationOnly.extend(list(date_lsd['Station']))

train_date_sample_stationcols = train_date_sample.copy()
train_date_sample_stationcols.columns = dateColsStationOnly

#----------------

featureColsStationOnly = ['Id']
featureColsStationOnly.extend(list(all_lsf['Station']))
featureColsStationOnly.append('Response')

train_feature_sample_stationcols = train_feature_sample.copy()
train_feature_sample_stationcols.columns = featureColsStationOnly

In [49]:
# it seems like there's duplicate station fields.  But they all have the same values... or do they?

for stationNum in train_date_sample_stationcols.columns.unique().drop('Id'):
    if len(train_date_sample_stationcols[stationNum].shape)==1:
        continue
    diff = (train_date_sample_stationcols[stationNum].max(axis=1)-train_date_sample_stationcols[stationNum].min(axis=1)).max()
    if round(diff, 5) != 0:
        print(stationNum, diff)
        
# Results:

    # Station 24 has 288 different Features (it's a big station), so there was probably a lot of time taken here
    # Station 25 has 333 different Features (it's a big station), so there was probably a lot of time taken here as well
    # Station 42 has no operations taken here (out of the first 10000 samples), so we have an NaN value

24 0.29
25 34.25
42 nan


In [50]:
# so now we know we can consolidate the station fields into 1, let's do that, and create a new DataFrame:

df = pd.DataFrame(columns=train_date_sample_stationcols.columns.unique())
df['Id'] = train_date_sample_stationcols['Id']

for stationNum in df.drop('Id', axis=1):
    if len(train_date_sample_stationcols[stationNum].shape)==1: #if only 1 column
        df[stationNum] = train_date_sample_stationcols[stationNum]
        continue
    df[stationNum] = train_date_sample_stationcols[stationNum].mean(axis=1)
    

train_date_sample_stationcols_unique = df

In [51]:
# append 'Response' to the back

train_date_sample_stationcols_unique[99] = train_numeric_sample['Response'].apply(lambda x: 999999 if x==0 else np.nan)
train_date_sample_stationcols_unique[100] = train_numeric_sample['Response'].apply(lambda x: 999999 if x==1 else np.nan)

In [52]:
train_date_sample_stationcols_unique.tail()

,Id,0,1,2,3,4,5,6,7,8,...,44,45,46,47,48,49,50,51,99,100
9995,53866,373.90,373.90,374.01,NaN,374.03,NaN,374.03,NaN,374.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,NaN
9996,53867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,NaN
9997,53869,676.59,676.59,676.59,NaN,676.61,NaN,676.62,NaN,676.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,NaN
9998,53872,533.03,533.03,NaN,533.04,NaN,533.05,533.06,NaN,533.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,NaN
9999,53873,533.03,533.03,NaN,533.04,NaN,533.05,533.06,NaN,533.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,NaN


In [53]:
# Creating a list of sizes, based on how frequently the stations are hit:
# nodesSizeList
import math

def getNodesSizeList(dfStationCols, scalar=1, islog=False):
    maximumValue = dfStationCols.shape[0]
    if islog:
        nodesSizeList = [math.log(dfStationCols[stationNum].dropna().size*scalar/maximumValue+1) 
                 for stationNum in dfStationCols.columns]
    else:
        nodesSizeList = [dfStationCols[stationNum].dropna().size*scalar/maximumValue+1 
                 for stationNum in dfStationCols.columns]
    
    return nodesSizeList
    
#-----------------------------------

nodesSizeList = getNodesSizeList(train_date_sample_stationcols_unique, scalar=50000, islog=True)
#nodesSizeList = getNodesSizeList(train_date_sample_stationcols_unique, scalar=5, islog=False)

In [54]:
# now we create connections, based on train_date_sample_stationcols_unique: we can see sequence of movement between stations:

def getConnections(inputSeries):
    '''
    Get a list of tuples for all connections, starting from -1 (Start), and ending at either 99 (Pass), or 100 (Fail)
    
    Input:  A Series with a lot of NaNs, and indexes are Station Names (integers)
    
    Output: A List of Tuples with all connections of Pairs of Adjacent Stations without NaNs
    '''
    droppedNa = list(inputSeries.dropna().drop('Id').index)
    listOfConnections = [(-1,droppedNa[0])]
    for i in range(0, len(droppedNa)-1):
        listOfConnections.append((droppedNa[i],droppedNa[i+1]))
    return listOfConnections    


def flattenConnectionsSeries(connectionsSeries):
    mylist = []
    for item in connectionsSeries:
        mylist.extend(item)
    return mylist

In [55]:
# Connection Frequency will list all the connections and their corresponding (not-normalized) frequencies
# Not too important yet... below will be the Normalized version which is VERY important

allConnections = train_date_sample_stationcols_unique.apply(getConnections, axis=1)
allConnections = flattenConnectionsSeries(allConnections)

from collections import Counter
connectionFrequency = Counter(allConnections)

#connectionFrequency

In [56]:
import math

def normalizeFrequencyDict(frequencyDict, scalar=1, islog=False):
    maximumValue = max(frequencyDict.values())
    newDict = {}
    
    for key in frequencyDict:
        if islog:
            newDict[key] = math.log(frequencyDict[key]*scalar/maximumValue)
        else:
            newDict[key] = frequencyDict[key]*scalar/maximumValue
    
    return newDict 


def convertFrequencyDictToFrequencyList(frequencyDict):
    return [(key[0], key[1], frequencyDict[key]) for key in frequencyDict.keys()]

#### Creating arguments to feed in Network.add_edges (incl. thickness)

In [57]:
# Normalized connection frequency
# EXTREMELY IMPORTANT - this is what the nodes-edges will be for the node graph

normConnectionFrequency = normalizeFrequencyDict(connectionFrequency, scalar=100, islog=True)

nodesEdgeList = convertFrequencyDictToFrequencyList(normConnectionFrequency)

#### Back to Nodes: Color coded 

Will be how much this station contributes to the FAIL vs PASS (use ratios/rates)

The Darker, the more this contributes to FAIL

In [58]:
# station_frequency_df is a Dataframe that shows how many times an entry is at each station

station_frequency_df = pd.DataFrame(train_feature_sample_stationcols['Id'])

for station in range(0,52):
    station_frequency_df[station] = pd.DataFrame(train_feature_sample_stationcols[station]).count(axis=1)#count number of not-NaNs in that row

station_frequency_df['Response'] = train_feature_sample['Response']

In [59]:
# Create Color Gradient for Nodes

import math

station_frequency_df_pass = station_frequency_df[station_frequency_df['Response']==0]
station_frequency_df_fail = station_frequency_df[station_frequency_df['Response']==1]

failrate = station_frequency_df_fail.sum(axis=0)/station_frequency_df_pass.sum(axis=0)
failrate.drop(['Id','Response'],inplace=True)
failrate.fillna(0, inplace=True)
    
#nodesColorGradient = [rate*255/failrate.max() for rate in failrate]
nodesColorGradient = [math.log(rate*1000+1)*255/math.log(failrate.max()*1000+1) for rate in failrate]

nodesColorList = ['orange']
nodesColorList.extend(['rgb({b},{d},0)'.format(b=gradient,d=255-gradient) for gradient in nodesColorGradient])
nodesColorList.extend(['green', 'red'])

## Now let's graph this Shop Layout!

Color:  The more Red, the higher the Fail Rate for items that pass through that Station

In [60]:
#node coordinates according to @JohnM in Kernal/Discussion shop floor

y=[0, #Start
   1,2,3,3,4,4,5,5,6,7,7,           #0 to 10
   7,1,2,3,3,4,4,5,5,6,               #11 to 20
   7,7,7,6,6,7,8,9,8,9,               #21 to 30
   10,10,10,11,12,12,15,17,10,11,       #31 to 40
   12,10,13,13,14,14,15,16,17,17,18,  #41 to 51
   19,19]  #Pass/Fail

x=[5, #Start
   1,1,2,0,2,0,2,0,1,2,1,         #0 to 10
   0,9,9,8,10,8,10,10,8,9,               #11 to 20
   9,8,10,4,6,5,6,3,5,4,               #21 to 30
   8,6,7,7,6,8,7,8,3,3,               #31 to 40
   3,2,4,2,3,2,3,3,4,2,3,  #41 to 51
   3,7]  #Pass/Fail

In [61]:
from pyvis import network as net
import networkx as nx #for graphing nodes

g = net.Network(height='1800px', width='1200px', notebook=False, layout=0) #creating node graph object

for i in range(0, len(nodesList)):
    g.add_node(nodesList[i], 
               label=nodesLabelList[i], 
               size=nodesSizeList[i], 
               color=nodesColorList[i], 
               y=y[i]*90, 
               x=x[i]*70,
              shape=nodesShapeList[i])

#g.add_nodes(nodesList, 
            #label=nodesLabelList, 
            #color=nodesColorList,
           #size=nodesSizeList) #creating nodes

nodesEdgeList.sort()

g.add_edges(nodesEdgeList) #creating edges (thickness is the 3rd optional argument)

g.toggle_drag_nodes(True)
g.toggle_physics(False)

g.show('mygraph2.html') #generating the graph and showing it

## Looks like Station 32 has the highest fail rate, and Station 31 has the highest pass rate.

Are there any combinations of stations that have super-high rates?

In [29]:
allCombos = 

SyntaxError: invalid syntax (<ipython-input-29-ec0a5f768b04>, line 1)

In [30]:
%%time
# Let's load our test data:

batch_size_test = 10000
test_num_records = 1183747
skiprows_test = np.random.randint(1, test_num_records+1, size=test_num_records-batch_size_test)

test_categorical_sample = pd.read_csv(folder+'test_categorical.csv', nrows=batch_size, skiprows=skiprows)
test_numeric_sample = pd.read_csv(folder+'test_numeric.csv', nrows=batch_size, skiprows=skiprows)
test_date_sample = pd.read_csv(folder+'test_date.csv', nrows=batch_size, skiprows=skiprows)


# merge categorical and numeric together for total train_features_sample DataFrame
test_feature_sample = pd.merge(test_categorical_sample, test_numeric_sample,on='Id')

<string>:2: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,43,44,46,57,59,60,62,63,65,66,68,69,71,72,74,75,77,78,80,81,83,84,86,87,89,90,92,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,324,335,337,340,342,344,346,348,351,354,357,360,363,366,369,372,375,378,381,384,387,390,393,396,399,402,407,410,413,416,419,422,425,428,431,434,437,440,443,446,449,452,455,458,461,464,467,470,473,476,479,482,485,488,491,494,497,500,503,506,509,512,515,518,549,611,614,617,620,623,626,629,632,635,638,641,644,647,650,653,656,659,662,665,668,671,674,677,680,683,686,689,692,695,698,701,704,707,710,713,716,719,722,807,809,811,813,815,817,820,823,826,829,832,835,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,89

CPU times: user 35.4 s, sys: 6.33 s, total: 41.8 s
Wall time: 41.8 s


In [31]:
# For reference: train_feature_sample_stationcols is the DF of feature codes with Station fields

train_feature_sample_stationcols.head()

,Id,1,1,1,1,2,2,2,2,2,...,50,50,50,50,50,51,51,51,51,Response
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [32]:
# Now let's create the test_feature_sample_stationcols:  the test version of the DF above^
# Note that this DF has 1 fewer column ('Response')

featureColsStationOnly = ['Id']
featureColsStationOnly.extend(list(all_lsf['Station']))

test_feature_sample_stationcols = test_feature_sample.copy()
test_feature_sample_stationcols.columns = featureColsStationOnly

In [33]:
test_feature_sample_stationcols

,Id,1,1,1,1,2,2,2,2,2,...,50,50,50,50,50,50,51,51,51,51
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
